<a href="https://colab.research.google.com/github/satyank/ais-workshops/blob/main/CLIP_Workshop_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIM CV Workshop

---
Go to file and click save a copy in drive

---




## Runtime & Installs
- Runtime -> Change runtime type ->  GPU
- Run once, if NumPy shows as 2.x, restart session and re-run from top.

In [17]:
!pip -q install clip-anytorch ftfy==6.3.1
import numpy as np, torch, clip
print("NumPy:", np.__version__, "| Torch:", torch.__version__)


NumPy: 2.0.2 | Torch: 2.8.0+cu126


## Don't edit
Code for the webcam


In [16]:
from IPython.display import display, Javascript, Image as IPyImage
from google.colab import output
import base64
from PIL import Image
import torch, clip

def _take_photo_impl(filename='photo.jpg', quality=0.9):
    js_code = f"""
        async function takePhoto(quality) {{
          const ID = 'webcam-div-unique';
          const old = document.getElementById(ID);
          if (old) old.remove();

          const div = document.createElement('div');
          div.id = ID;

          const btn = document.createElement('button');
          btn.textContent = 'Capture';
          btn.style.fontSize = '16px';
          btn.style.padding = '8px 14px';
          btn.style.marginBottom = '8px';

          const video = document.createElement('video');
          video.style.display = 'block';
          video.style.maxWidth = '100%';

          div.appendChild(btn);
          div.appendChild(video);
          document.body.appendChild(div);

          let stream = null;
          try {{
            stream = await navigator.mediaDevices.getUserMedia({{ video: true }});
          }} catch (e) {{
            alert('Ur camera is blocked. Allow access in URL bar at the top.');
            return null;
          }}

          video.srcObject = stream;
          await video.play();

          google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

          await new Promise((resolve) => btn.onclick = resolve);

          const canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          canvas.getContext('2d').drawImage(video, 0, 0);

          if (stream) stream.getVideoTracks().forEach(t => t.stop());
          div.remove();

          return canvas.toDataURL('image/jpeg', quality);
        }}
    """
    display(Javascript(js_code))

    data = output.eval_js(f'takePhoto({quality})')
    if not data:
        return None

    header, encoded = data.split(',', 1)
    binary = base64.b64decode(encoded)
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

def take_photo():
    fname = _take_photo_impl()
    if fname:
        display(IPyImage(fname))
    return fname




## Action detection


In [18]:
#function
def recognize_zero_shot(image_path, actions, model, preprocess, device):
  image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
  # Normalize features
  with torch.no_grad():
    image_features = model.encode_image(image)
    text_inputs = clip.tokenize(actions).to(device)
    text_features = model.encode_text(text_inputs)

  # Compute similarity and pick best label
  image_features /= image_features.norm(dim=-1, keepdim=True)
  text_features /= text_features.norm(dim=-1, keepdim=True)

  similarity = image_features @ text_features.T
  pred_index = similarity.argmax().item()
  return action[pred_index]


## Imports


In [19]:
#write your imports here
import numpy as np
import cv2
from PIL import Image
import torch
import clip

## Load CLIP


In [20]:
#pick device and load model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
clip_model, preprocess = clip.load('ViT-B/32', device = device)
clip_model.eval()
print('Device', device)

Device cuda


## Define Actions
Make a short list of actions to test.

In [21]:
ACTIONS = [
    #action list
    'A person waving',
    'A person sitting still'
    'A person doing a peace sign'
]
print('ACTIONS:', ACTIONS)

ACTIONS: ['A person waving', 'A person sitting stillA person doing a peace sign']


## Capture


In [23]:
#our function is called take_photo()
filename = take_photo()
print('Captured:', ACTIONS)


<IPython.core.display.Javascript object>

KeyboardInterrupt: 

## Predict


In [24]:
if filename is None:
  print('Photo not captured')
else
  # best = recognize_zero_shot.
  print('Photo captured')


SyntaxError: expected ':' (ipython-input-2751498639.py, line 3)

## Try new actions
